In [17]:
from types import SimpleNamespace

args = SimpleNamespace(
    input_files=["task1_test_gpt35.json"],
    skip_response_quality=False,
    max_rows=40
)

In [12]:
import sys
sys.path.append(f'../source')

import evaluation
import pandas as pd
from tqdm.notebook import tqdm
import os

In [13]:
for file in args.input_files:
    input_file = f'../data/{file}'
    output_file = f'../data/{file.replace(".json", "")}_eval.json'
    if not os.path.exists(output_file):
        testset = pd.read_json(input_file)
        testset['Distinctiveness'] = [None] * len(testset)
        testset['positive_constraints'] = [[]] * len(testset)
        for quality_metric in evaluation.gpt_metrics.keys():
            testset[quality_metric] = [None] * len(testset)
    else: 
        testset = pd.read_json(output_file)
            
    condition = (testset['responses'].apply(len)>0) & testset['Relevance'].isna()
    max_rows = min(args.max_rows, len(testset))
    subset = testset[condition]


In [16]:

for idx, case in tqdm(subset.iterrows(), total=max(max_rows-(~condition).sum(), 0)):
    if idx >= max_rows: break
    metrics = evaluation.evaluate(case['context'], case['responses'], case['constraints'], evaluate_quality=not args.skip_response_quality)
    print(metrics)
    for metric, value in metrics.items():
        testset.at[idx, metric] = value
    #testset.to_json(output_file)

0it [00:00, ?it/s]

In [18]:
testset['Distinctiveness']

0      0.708333
1      0.600000
2      0.714286
3      0.962963
4      0.670330
         ...   
295    1.000000
296    0.940299
297    0.765957
298    0.935065
299    0.942029
Name: Distinctiveness, Length: 300, dtype: float64